In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2024-12-25 08:48:22.324305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735116502.340267    3833 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735116502.344809    3833 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 08:48:22.363154: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# Load dataset paths and labels
data_dir = "./data/"
train_csv = pd.read_csv(os.path.join(data_dir, "train.csv"))


In [3]:
# Prepare labels
train_csv["label"] = (
    train_csv["healthy"] * 0 +
    train_csv["multiple_diseases"] * 1 +
    train_csv["rust"] * 2 +
    train_csv["scab"] * 3
)
y = train_csv["label"].values
y_encoded = to_categorical(y, num_classes=4)


In [7]:
# Load and preprocess images
def load_data(df, data_dir, image_size):
    images = []
    for _, row in df.iterrows():
        img_path = os.path.join(data_dir, "images", row['image_id'] + ".jpg")
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
        img = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img)
    return np.array(images)

In [8]:

# Resize images to 224x224 for VGG19
image_size = (224, 224)
X = load_data(train_csv, data_dir, image_size)
X = X / 255.0  # Normalize pixel values

In [9]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [10]:
# Data augmentation
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [11]:
# Load the VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


I0000 00:00:1735116702.594905    3833 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 22s 0us/step


In [12]:
# Freeze the base model layers
base_model.trainable = False

In [13]:

# Add custom layers for the new classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)  # 4 classes

In [14]:
# Define the final model
model = Model(inputs=base_model.input, outputs=output)


In [15]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [17]:
# Train the model
history = model.fit(
    data_gen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=[early_stopping]
)

/home/tanuj/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1735116758.241733    4017 service.cc:148] XLA service 0x7fb968002f00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735116758.241801    4017 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-12-25 08:52:38.273829: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1735116758.560799    4017 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-12-25 08:52:39.120883: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,224,224]{3,2,1,0}, f32[64,3,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_c

19/46 ━━━━━━━━━━━━━━━━━━━━ 7s 277ms/step - accuracy: 0.3083 - loss: 1.5142

2024-12-25 08:52:57.250385: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[16,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,3,224,224]{3,2,1,0}, f32[64,3,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-12-25 08:52:57.360008: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[16,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasAct

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step - accuracy: 0.3272 - loss: 1.4225

2024-12-25 08:53:16.704479: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,3,224,224]{3,2,1,0}, f32[64,3,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-12-25 08:53:16.818611: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasAct

46/46 ━━━━━━━━━━━━━━━━━━━━ 46s 695ms/step - accuracy: 0.3276 - loss: 1.4205 - val_accuracy: 0.3370 - val_loss: 1.2113
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 16s 350ms/step - accuracy: 0.4216 - loss: 1.2133 - val_accuracy: 0.4658 - val_loss: 1.1723
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 16s 353ms/step - accuracy: 0.4600 - loss: 1.1952 - val_accuracy: 0.5068 - val_loss: 1.1384
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 16s 351ms/step - accuracy: 0.4705 - loss: 1.1539 - val_accuracy: 0.5096 - val_loss: 1.1201
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 16s 353ms/step - accuracy: 0.5003 - loss: 1.1375 - val_accuracy: 0.5781 - val_loss: 1.0821
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 16s 351ms/step - accuracy: 0.5224 - loss: 1.0974 - val_accuracy: 0.5753 - val_loss: 1.0791
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 16s 351ms/step - accuracy: 0.5327 - loss: 1.0981 - val_accuracy: 0.5589 - val_loss: 1.0572
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 16s 350ms/step - accuracy: 0.5849 - loss: 1.0406 - val_accuracy: 0.580

In [18]:
# Save the model
model.save("vgg19_crop_disease_model.keras")

In [19]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=2)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

12/12 - 3s - 272ms/step - accuracy: 0.6164 - loss: 0.9769
Validation Accuracy: 61.64%


In [20]:
base_model.trainable = True
for layer in base_model.layers[:-5]:  # Freeze all layers except the last 5
    layer.trainable = False

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
history = model.fit(
    data_gen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20


2024-12-25 08:59:21.458145: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,512,14,14]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,512,14,14]{3,2,1,0}, f32[512,512,3,3]{3,2,1,0}, f32[512]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}


44/46 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.4661 - loss: 1.2537

2024-12-25 08:59:40.682687: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[16,512,14,14]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,512,14,14]{3,2,1,0}, f32[512,512,3,3]{3,2,1,0}, f32[512]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}


46/46 ━━━━━━━━━━━━━━━━━━━━ 29s 513ms/step - accuracy: 0.4700 - loss: 1.2439 - val_accuracy: 0.6055 - val_loss: 0.9916
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 20s 427ms/step - accuracy: 0.7106 - loss: 0.7933 - val_accuracy: 0.6055 - val_loss: 1.2089
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 20s 427ms/step - accuracy: 0.7343 - loss: 0.7768 - val_accuracy: 0.7342 - val_loss: 0.7438
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 20s 427ms/step - accuracy: 0.7811 - loss: 0.6367 - val_accuracy: 0.7644 - val_loss: 0.7227
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 20s 432ms/step - accuracy: 0.7947 - loss: 0.5644 - val_accuracy: 0.7205 - val_loss: 0.8750
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 20s 428ms/step - accuracy: 0.8149 - loss: 0.5656 - val_accuracy: 0.7699 - val_loss: 0.6854
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 20s 430ms/step - accuracy: 0.8179 - loss: 0.5222 - val_accuracy: 0.8000 - val_loss: 0.6086
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 20s 435ms/step - accuracy: 0.8554 - loss: 0.4388 - val_accuracy: 0.769

In [23]:
model.save("vgg19_trainable_crop_disease_model.keras")

In [24]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=2)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

12/12 - 3s - 269ms/step - accuracy: 0.8000 - loss: 0.6086
Validation Accuracy: 80.00%


In [25]:
# Define class labels (mapping indices to class names)
class_labels = {
    0: "healthy",
    1: "multiple_diseases",
    2: "rust",
    3: "scab"
}

In [30]:
# Load a test image
test_image_path = "./data/images/Train_33.jpg"
img = tf.keras.preprocessing.image.load_img(test_image_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]  # Get the index of the predicted class

# Map index to class label
predicted_class_label = class_labels[predicted_class_index]

print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted class label: {predicted_class_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
Predicted class index: 0
Predicted class label: healthy
